<a href="https://colab.research.google.com/github/shreya-k17/MovieRecommendation/blob/main/Movie_Recommendation_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import difflib
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer


In [ ]:
#Data Collection and pre-processing
#loading the dataframe from csv file to the dataframe
movies_data= pd.read_csv('/movies.csv')

<ipython-input-9-d8bed07ef8e6>:3: DtypeWarning: Columns (0,1,4,9,13,14,19,20,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

In [ ]:
#printing first five rows
movies_data.head()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,Unnamed: 1255,Unnamed: 1256,Unnamed: 1257,Unnamed: 1258,Unnamed: 1259,Unnamed: 1260,Unnamed: 1261,Unnamed: 1262,Unnamed: 1263,Unnamed: 1264
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.31295,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
#number of rows and columns in the dataframe
movies_data.shape

(4809, 1265)

In [ ]:
#selecting the relavent features for recommendation
selected_features=['genres','keywords','tagline','cast','director']
print(selected_features)

['genres', 'keywords', 'tagline', 'cast', 'director']


In [ ]:
for feature in selected_features:
  movies_data[feature]=movies_data[feature].fillna

In [ ]:
df=pd.DataFrame(movies_data)

In [ ]:
#combining all the five selected features
combined_features= df['genres'],' ',df['keywords'],' ',df['tagline'],' ',df['cast'],' ',df['director']

In [ ]:
data=pd.read_csv('/movies.csv')

<ipython-input-17-eeb9cbf04d1a>:1: DtypeWarning: Columns (0,1,4,9,13,14,19,20,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,27

In [ ]:
df=pd.DataFrame(data)

In [ ]:
df["genres"]

0       Action Adventure Fantasy Science Fiction
1                       Adventure Fantasy Action
2                         Action Adventure Crime
3                    Action Crime Drama Thriller
4               Action Adventure Science Fiction
                          ...                   
4804                       Action Crime Thriller
4805                              Comedy Romance
4806               Comedy Drama Romance TV Movie
4807                                         NaN
4808                                 Documentary
Name: genres, Length: 4809, dtype: object

In [ ]:

combined_features= df['genres']+' '+df['keywords']+' '+df['tagline']+' '+df['cast']+' '+df['director']

In [ ]:
print(combined_features)

0       Action Adventure Fantasy Science Fiction cultu...
1       Adventure Fantasy Action ocean drug abuse exot...
2       Action Adventure Crime spy based on novel secr...
3       Action Crime Drama Thriller dc comics crime fi...
4       Action Adventure Science Fiction based on nove...
                              ...                        
4804    Action Crime Thriller united states\u2013mexic...
4805                                                  NaN
4806                                                  NaN
4807                                                  NaN
4808                                                  NaN
Length: 4809, dtype: object


In [ ]:
#converting the text data into feature vector

vectorizer= TfidfVectorizer()

In [ ]:
combined_features.head()

0    Action Adventure Fantasy Science Fiction cultu...
1    Adventure Fantasy Action ocean drug abuse exot...
2    Action Adventure Crime spy based on novel secr...
3    Action Crime Drama Thriller dc comics crime fi...
4    Action Adventure Science Fiction based on nove...
dtype: object

In [ ]:
combined_features = combined_features.fillna('')


In [ ]:
feature_vectors = vectorizer.fit_transform(combined_features)


In [ ]:
print(feature_vectors)

  (0, 2064)	0.1707819490145591
  (0, 6664)	0.11738803662518522
  (0, 11117)	0.1961103238738005
  (0, 8728)	0.16328378260778725
  (0, 7472)	0.23469394291398177
  (0, 12483)	0.15426312438863335
  (0, 14243)	0.19495675537512416
  (0, 12004)	0.20119633464898123
  (0, 11362)	0.21929381087585958
  (0, 14777)	0.20725653754804918
  (0, 14555)	0.23095790769065858
  (0, 11389)	0.15126848339775434
  (0, 9808)	0.26580549150751664
  (0, 9537)	0.09152409107612518
  (0, 14546)	0.12857562599451414
  (0, 13046)	0.07132520833916563
  (0, 4241)	0.23469394291398177
  (0, 12190)	0.2127075654438508
  (0, 2748)	0.24960228777167906
  (0, 14169)	0.12885633258526735
  (0, 12282)	0.33846868536334
  (0, 5019)	0.16328378260778725
  (0, 2607)	0.2218327365672547
  (0, 3156)	0.21694313284525513
  (0, 4675)	0.1053559323086701
  :	:
  (4804, 6652)	0.16278077437453972
  (4804, 8315)	0.16739415871492583
  (4804, 7618)	0.18603730315623238
  (4804, 2224)	0.15021582974977463
  (4804, 3625)	0.15021582974977463
  (4804, 7265)

In [ ]:
#COSINE SIMILARITY
#getting the similarity score using cosine similarity

similarity= cosine_similarity(feature_vectors)



In [ ]:
print(similarity)

[[1.         0.0741602  0.03929261 ... 0.         0.         0.        ]
 [0.0741602  1.         0.03397071 ... 0.         0.         0.        ]
 [0.03929261 0.03397071 1.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]]


In [ ]:
print(similarity.shape)

(4809, 4809)


In [ ]:
#getting the movie name  from the user

movie_name= input('ENTER YOUR FAVORITE MOVIE NAME---')

ENTER YOUR FAVORITE MOVIE NAME---iron man


In [ ]:
if not isinstance(movie_name, str):
    movie_name = str(movie_name)

In [ ]:
#creating a list of all the movie names given in the dataset

In [ ]:
list_all_titles = [str(title) for title in list_all_titles]
print(list_all_titles)

['Avatar', "Pirates of the Caribbean: At World's End", 'Spectre', 'The Dark Knight Rises', 'John Carter', 'Spider-Man 3', 'Tangled', 'Avengers: Age of Ultron', 'Harry Potter and the Half-Blood Prince', 'Batman v Superman: Dawn of Justice', 'Superman Returns', 'Quantum of Solace', "Pirates of the Caribbean: Dead Man's Chest", 'The Lone Ranger', 'Man of Steel', 'The Chronicles of Narnia: Prince Caspian', 'The Avengers', 'Pirates of the Caribbean: On Stranger Tides', 'Men in Black 3', 'The Hobbit: The Battle of the Five Armies', 'The Amazing Spider-Man', 'Robin Hood', 'The Hobbit: The Desolation of Smaug', 'The Golden Compass', 'King Kong', 'Titanic', 'Captain America: Civil War', 'Battleship', 'Jurassic World', " 'department': 'Sound'", 'Skyfall', 'Spider-Man 2', 'Iron Man 3', 'Alice in Wonderland', 'X-Men: The Last Stand', 'Monsters University', 'Transformers: Revenge of the Fallen', 'Transformers: Age of Extinction', 'Oz: The Great and Powerful', 'The Amazing Spider-Man 2', 'TRON: Lega

In [ ]:
if type(movie_name) == float:
    movie_name = str(movie_name)

In [ ]:
#finding the close match for the movie name given by the user
find_close_match=difflib.get_close_matches(movie_name, list_all_titles)

In [ ]:
print(find_close_match)

['Iron Man', 'Iron Man 3', 'Iron Man 2']


In [ ]:
close_match=find_close_match[0]
print(close_match)

Iron Man


In [ ]:
#finding the index of the movie with title

index_movie= movies_data[movies_data.title == close_match]['index'].values[0]
print(index_movie)

68


In [ ]:
print(f"index_movie: {index_movie}")

index_movie: 68


In [ ]:
print(f"index_movie: {index_movie}")

index_movie: 68


In [ ]:
print(f"similarity: {similarity}")

similarity: [[1.         0.0741602  0.03929261 ... 0.         0.         0.        ]
 [0.0741602  1.         0.03397071 ... 0.         0.         0.        ]
 [0.03929261 0.03397071 1.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]]


In [ ]:
similarity_score= list(enumerate(similarity[68]))

In [ ]:
print(similarity_score)

[(0, 0.031963670583476225), (1, 0.015417408425577735), (2, 0.009093995913303636), (3, 0.0), (4, 0.03103677566270209), (5, 0.009228810657478988), (6, 0.038770711684772904), (7, 0.03245214003547307), (8, 0.020360202550936977), (9, 0.008742877330019288), (10, 0.0), (11, 0.00781909634570662), (12, 0.036750838769203896), (13, 0.008147698861690904), (14, 0.03186097500873751), (15, 0.0188022238268354), (16, 0.030896100733511213), (17, 0.00867257225962547), (18, 0.07767315151140486), (19, 0.0077313962692791855), (20, 0.008863553303217102), (21, 0.007483617847534562), (22, 0.00806007054455083), (23, 0.0076989975703468565), (24, 0.00827228922825927), (25, 0.0), (26, 0.033298936576296685), (27, 0.0547718019070358), (28, 0.0), (29, 0.0), (30, 0.00924708340820954), (31, 0.008946304693464184), (32, 0.03165061774506413), (33, 0.018224029771663312), (34, 0.06263490600432016), (35, 0.048793041528855066), (36, 0.03406387715051313), (37, 0.1937525927061203), (38, 0.020662717149733556), (39, 0.00903681716

In [ ]:
len(similarity_score)

4809

In [ ]:
#SORTING THE MOVIES BASED ON THEIR SIMILARITY SCORE
sorted_similar_movies= sorted(similarity_score,key= lambda x:x[1], reverse=True)

In [ ]:
#print the name of similar movies based on the index
print('Movies suggested for you: ')
i=1
for movie in sorted_similar_movies:
  index=movie[0]
  title_from_index=movies_data[movies_data.index==index]['title'].values
  if(i<30):
    print(i,',',title_from_index)
    i+=1

Movies suggested for you: 
1 , ['Monsters vs Aliens']
2 , ['Pacific Rim']
3 , ['Sausage Party']
4 , ['Zathura: A Space Adventure']
5 , ['Transformers: Age of Extinction']
6 , ['Aliens in the Attic']
7 , ['Freeway']
8 , ['How Do You Know']
9 , ['Penguins of Madagascar']
10 , ['This Is the End']
11 , ['The 40 Year Old Virgin']
12 , ['The Iron Giant']
13 , ['Knocked Up']
14 , ['Titan A.E.']
15 , ['Stuart Little 2']
16 , ['Valiant']
17 , ['The Interview']
18 , ['Mars Needs Moms']
19 , ['Planet 51']
20 , ["Madagascar 3: Europe's Most Wanted"]
21 , ['Home']
22 , ['Real Steel']
23 , ['Jack the Giant Slayer']
24 , ['Paul']
25 , ['Four Christmases']
26 , ['Arthur Christmas']
27 , ['Dark City']
28 , ['Stuart Little']
29 , ['Tomorrowland']


In [ ]:
movie_name= input('ENTER YOUR FAVORITE MOVIE NAME---')

if not isinstance(movie_name, str):
    movie_name = str(movie_name)

list_all_titles = [str(title) for title in list_all_titles]

find_close_match=difflib.get_close_matches(movie_name, list_all_titles)

close_match=find_close_match[0]

index_movie= movies_data[movies_data.title == close_match]['index'].values[0]
similarity_score= list(enumerate(similarity[68]))
sorted_similar_movies= sorted(similarity_score,key= lambda x:x[1], reverse=True)

print('Movies suggested for you: ')
i=1
for movie in sorted_similar_movies:
  index=movie[0]
  title_from_index=movies_data[movies_data.index==index]['title'].values
  if(i<30):
    print(i,',',title_from_index)
    i+=1

ENTER YOUR FAVORITE MOVIE NAME---avatar
Movies suggested for you: 
1 , ['Monsters vs Aliens']
2 , ['Pacific Rim']
3 , ['Sausage Party']
4 , ['Zathura: A Space Adventure']
5 , ['Transformers: Age of Extinction']
6 , ['Aliens in the Attic']
7 , ['Freeway']
8 , ['How Do You Know']
9 , ['Penguins of Madagascar']
10 , ['This Is the End']
11 , ['The 40 Year Old Virgin']
12 , ['The Iron Giant']
13 , ['Knocked Up']
14 , ['Titan A.E.']
15 , ['Stuart Little 2']
16 , ['Valiant']
17 , ['The Interview']
18 , ['Mars Needs Moms']
19 , ['Planet 51']
20 , ["Madagascar 3: Europe's Most Wanted"]
21 , ['Home']
22 , ['Real Steel']
23 , ['Jack the Giant Slayer']
24 , ['Paul']
25 , ['Four Christmases']
26 , ['Arthur Christmas']
27 , ['Dark City']
28 , ['Stuart Little']
29 , ['Tomorrowland']
